In [58]:
from dotenv import load_dotenv
load_dotenv()
import os
from pinecone import Pinecone, ServerlessSpec
import google.generativeai as genai

In [48]:
pc = Pinecone(api_key = os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", dimension=768, metric="cosine", spec = ServerlessSpec(cloud="aws", region="us-east-1")
)

In [55]:
import json
data = json.load(open("reviews.json"))
data['reviews']


[{'professor': 'John Doe',
  'subject': 'Introduction to Computer Science',
  'stars': 4,
  'review': 'Professor Doe was a great teacher. He explained concepts clearly and was always willing to answer questions.'},
 {'professor': 'Jane Smith',
  'subject': 'Calculus I',
  'stars': 3,
  'review': 'The course was challenging, but Professor Smith was a good lecturer. She provided helpful examples and practice problems.'},
 {'professor': 'David Lee',
  'subject': 'English Composition',
  'stars': 5,
  'review': 'Professor Lee is an amazing teacher. He is passionate about literature and inspired me to love reading.'},
 {'professor': 'Emily Chen',
  'subject': 'Biology I',
  'stars': 2,
  'review': "The course material was too much for me. Professor Chen's lectures were difficult to follow."},
 {'professor': 'Thomas Brown',
  'subject': 'History of the United States',
  'stars': 4,
  'review': 'Professor Brown is a knowledgeable historian. He made the course interesting and engaging.'},
 {'p

In [68]:
processed_data = []
GOOGLE_API_KEY = os.environ['GEMINI_API_KEY']
genai.configure(api_key = GOOGLE_API_KEY)
for review in data['reviews']:
    result = genai.embed_content(
    model="models/text-embedding-004", content=review['review'], output_dimensionality=10
    )
    embedding = result["embedding"]
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }
    })

KeyError: 'GEMINI_API_KEY'